In [1]:
from time import sleep
import re
from bs4 import BeautifulSoup
import urllib.request
import urllib.parse
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100#&date=%2000:00:00&page=1'
html = urllib.request.urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

soup.find_all('div', class_ = ['section_body'])

[<div class="section_body" id="section_body"></div>]

In [7]:
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# path = 'C:/tool/chromedriver.exe'
# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# driver = webdriver.Chrome(path,options=options)

In [3]:
from selenium.common.exceptions import NoSuchElementException
path = 'C:/tool/chromedriver.exe'
driver = webdriver.Chrome(path)


for page in range(1, 21):
    url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={cat}#&date=%2000:00:00&page={page}'
    driver.get(url)
    sleep(5)
    for i in range(1, 5):
        for j in range(1, 6):
            # 경로가 계속 안맞아서 try.except문을 사용해서 예외는 그냥 pass 시켜버림
            # 100 부분이 크롤링되고 101 부분 넘어갈때 게속 오류가 발생
            try :
                driver.find_element_by_xpath(f'//*[@id="section_body"]/ul[{i}]/li[{j}]/dl/dt[2]/a').send_keys(Keys.ENTER)
                sleep(0.7)
                article = driver.find_element_by_xpath('//*[@id="articleBodyContents"]')
                a = re.sub('[^a-zA-Z가-힣\s]+', '', article.text)
                b = re.sub('[\n]+', ' ', a)
                driver.back()
                sleep(0.7)
            except NoSuchElementException:
                pass
driver.close()

In [4]:
len(article_politics)
len(article_economy)
len(article_society)
len(article_science)

399

In [5]:
import pandas as pd
df_politics = pd.DataFrame({'document':article_politics,'label':0})
df_economy = pd.DataFrame({'document':article_economy,'label':1})
df_society = pd.DataFrame({'document':article_society,'label':2})
df_science = pd.DataFrame({'document':article_science,'label':3})
df_politics.head()

,document,label
0,윤석열 당선인 시진핑 중국 국가주석과 오늘 통화 현직 대통령 아닌 당선인으론 첫 사...,0
1,정부 운영하려면 자기들 필요한 일인데 대소변 구분 못해 갈등엔 국민 짜증나밑에 사...,0
2,강기정 당대표가 공천 신청한 후보들 쥐잡듯이 잡아그것은 잘못된 것 이준석 국민의힘 ...,0
3,핵심요약 김경수 전 지사 낙마로 경남도 개월 권한대행 체제 경남지사 선거전 민주당 ...,0
4,김동연 유약설 출마 지역 너무 뜸들이다 보니 이재명 대장동 하자없다부산저축은행 더 ...,0


In [6]:
df = pd.concat([df_politics,df_economy,df_society,df_science])
df.shape

(1597, 2)

In [7]:
X = df['document']
y = df['label']

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [9]:
# 토큰화
import konlpy
from konlpy.tag import Okt
okt = Okt()
def okt_tokenizer(text):
    # 입력 인자로 들어온 text를 형태소 단어로 토큰화하여 list 객체 반환
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [10]:
# 정확도 검증
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

tfidf_vect = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2),min_df=3, max_df=0.9)
tfidf_vect.fit(X_train)
tfidf_maxrix_train = tfidf_vect.transform(X_train)

In [11]:
lg_clf = LogisticRegression(random_state=0)

params = {'C':[10,10.5,12]}

grid_cv = GridSearchCV(lg_clf,param_grid=params,cv=3,scoring='accuracy')
grid_cv.fit(tfidf_maxrix_train,y_train)
print(grid_cv.best_params_,round(grid_cv.best_score_,4))

{'C': 10} 0.8442


In [12]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용해 테스트 데이터를 TF-IDF값으로 피처 변환함.
tfidf_matrix_test = tfidf_vect.transform(X_test)

# Classifier는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
preds = grid_cv.best_estimator_.predict(tfidf_matrix_test)

print(f'Logistic Regression 정확도: {accuracy_score(y_test,preds)}')

Logistic Regression 정확도: 0.825
